# Mini Project - Modified ResNet Architecture with CIFAR-10 images

**Goal:** In this mini-project we are tasked with coming up with a modified residual network (ResNet)
architecture with the **highest test accuracy** on the CIFAR-10 image classification dataset, under the
constraint that our model has **no more than 5 million parameters**.

## Dataset loading and importing packages

We are using **Amazon SageMaker Studio Lab** for this mini-project and so there are a lot of packages to download. Additionally, we would be using **CIFAR-10 image classification dataset** which has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size. This dataset can be loaded using the **torchvision** package.

In [4]:
# Installing Packages - Pytorch and Torchvision
%pip install torch==1.10.0+cu111 torchvision==0.11.1+cu111 -f https://download.pytorch.org/whl/torch_stable.html --upgrade

Looking in links: https://download.pytorch.org/whl/torch_stable.html
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Importing packages
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

import random

**Training, Validation and Testing Dataloaders** - The training and validation dataloaders are split from the train directory using random seed shuffling of indices. The training dataloader then uses train transformations and batch size 128 which the validation dataloader uses the test transformations with batch size 100. The testing dataset comes from test directory, and dataloader uses test transformations and batch size 100.

In [6]:
# Loading and Transforming dataset
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Train dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)

# trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)

# Validation dataset
valid_dataset = torchvision.datasets.CIFAR10(
        root='./data', train=True,
        download=True, transform=transform_test,
    )

# Train - validation split
num_train = len(trainset)
indices = list(range(num_train))
# Size of validation dataset
valid_size = 0.1
split = int(np.floor(valid_size * num_train))
# Shuffle the dataset indices
random_seed = random.seed(30)
np.random.seed(random_seed)
np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# Dataloaders for train and validation dataset
train_loader = torch.utils.data.DataLoader(
        trainset, batch_size=128, sampler=train_sampler,
        num_workers=2
    )
valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=100, sampler=valid_sampler,
        num_workers=2
    )

    
# Test dataset and loader
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

# Target classes
classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [7]:
# Define our device as the first visible cuda device if we have CUDA available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


## Define a ResNet

In [229]:
# Installing and Importing Libraries
%pip install torchsummary 
from torchsummary import summary

Note: you may need to restart the kernel to use updated packages.


In [230]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [231]:
class BasicBlockKernel(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlockKernel, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=1, stride=stride, padding=0, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=1,
                               stride=1, padding=0, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [232]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

Created a custom Resnet function to test all configurations

In [233]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_channels = [64, 128, 256, 512], strides = [1, 2, 2, 2], linear_in= 512, in_planes = 64, avg_pool_size = 4, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = in_planes

        self.conv1 = nn.Conv2d(3, in_planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_planes)
        self.layers_list = []
        for i in range(len(num_blocks)):
            # Makes layers based on the num_blocks, num_channels, strides given
            self.layers_list.append(self._make_layer(block, num_channels[i], num_blocks[i], stride=strides[i]))
        self.layer1 = nn.Sequential(*self.layers_list)
        self.linear = nn.Linear(linear_in*block.expansion, num_classes)
        self.avg_pool_size = avg_pool_size

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = F.avg_pool2d(out, self.avg_pool_size)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

## Testing different configurations of Resnet-18

### Setting up different configurations

Experimented with:

Configuration 1 - Reducing number of blocks

Configuration 2 - Reducing number of layers and blocks with bottleneck

Configuration 3 - Reducing number of layers, blocks and increasing number of channels

Configuration 4 - Reducing number of layers, blocks, increasing number of channels and modifying average pool kernel size 

Configuration 5 - Changing number of blocks, kernel size of a block in  the residual layer and avg pool kernel size

Configuration 6 - Same configuration as configuration 3 but changing number of in planes

Configuration 7 - Same configuration as configuration 4 but changing number of in planes

In [149]:
# Original Resnet-18
net = ResNet(BasicBlock, [2, 2, 2, 2], [64, 128, 256, 512], [1, 2, 2, 2])
net = net.to(device)
print(summary(net, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [150]:
# Reducing number of residual blocks
net1 = ResNet(BasicBlock, [2, 1, 1, 1], [64, 128, 256, 512], [1, 2, 2, 2])
net1 = net1.to(device)
print(summary(net1, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]          36,864
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
        BasicBlock-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]          36,864
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]          36,864
      BatchNorm2d-11           [-1, 64, 32, 32]             128
       BasicBlock-12           [-1, 64, 32, 32]               0
           Conv2d-13          [-1, 128, 16, 16]          73,728
      BatchNorm2d-14          [-1, 128,

In [151]:
# residual layer reduction and residual block modification with bottleneck
net2 = ResNet(Bottleneck, [3, 3, 3], [64, 128, 256], [1, 2, 2], 1024)
net2 = net2.to(device)
print(summary(net2, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]           4,096
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]          36,864
       BatchNorm2d-6           [-1, 64, 32, 32]             128
            Conv2d-7          [-1, 256, 32, 32]          16,384
       BatchNorm2d-8          [-1, 256, 32, 32]             512
            Conv2d-9          [-1, 256, 32, 32]          16,384
      BatchNorm2d-10          [-1, 256, 32, 32]             512
       Bottleneck-11          [-1, 256, 32, 32]               0
           Conv2d-12           [-1, 64, 32, 32]          16,384
      BatchNorm2d-13           [-1, 64, 32, 32]             128
           Conv2d-14           [-1, 64,

In [234]:
# reducing residual layer, reducing residual block and increasing number of chan modification
net3 = ResNet(BasicBlock, [1, 1, 1], [128, 256, 512], [1, 2, 2], 2048)
net3 = net3.to(device)
print(summary(net3, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 32, 32]          73,728
       BatchNorm2d-4          [-1, 128, 32, 32]             256
            Conv2d-5          [-1, 128, 32, 32]         147,456
       BatchNorm2d-6          [-1, 128, 32, 32]             256
            Conv2d-7          [-1, 128, 32, 32]           8,192
       BatchNorm2d-8          [-1, 128, 32, 32]             256
        BasicBlock-9          [-1, 128, 32, 32]               0
           Conv2d-10          [-1, 256, 16, 16]         294,912
      BatchNorm2d-11          [-1, 256, 16, 16]             512
           Conv2d-12          [-1, 256, 16, 16]         589,824
      BatchNorm2d-13          [-1, 256, 16, 16]             512
           Conv2d-14          [-1, 256,

In [235]:
# reducing residual layer, reducing residual block, increasing number of channels, modifying average pool kernel size 
net4 = ResNet(BasicBlock, [1, 1, 1], [128, 256, 512], [1, 2, 2], 8192, avg_pool_size = 2)
net4 = net4.to(device)
print(summary(net4, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3          [-1, 128, 32, 32]          73,728
       BatchNorm2d-4          [-1, 128, 32, 32]             256
            Conv2d-5          [-1, 128, 32, 32]         147,456
       BatchNorm2d-6          [-1, 128, 32, 32]             256
            Conv2d-7          [-1, 128, 32, 32]           8,192
       BatchNorm2d-8          [-1, 128, 32, 32]             256
        BasicBlock-9          [-1, 128, 32, 32]               0
           Conv2d-10          [-1, 256, 16, 16]         294,912
      BatchNorm2d-11          [-1, 256, 16, 16]             512
           Conv2d-12          [-1, 256, 16, 16]         589,824
      BatchNorm2d-13          [-1, 256, 16, 16]             512
           Conv2d-14          [-1, 256,

In [236]:
# changing number of residual blocks, kernel size of a block in residual layer and avg pool kernel size
net5 = ResNet(BasicBlockKernel, [8, 7, 7, 7], [64, 128, 256, 512], [1, 2, 2, 2], 8192, avg_pool_size = 1)
net5 = net5.to(device)
print(summary(net5, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,728
       BatchNorm2d-2           [-1, 64, 32, 32]             128
            Conv2d-3           [-1, 64, 32, 32]           4,096
       BatchNorm2d-4           [-1, 64, 32, 32]             128
            Conv2d-5           [-1, 64, 32, 32]           4,096
       BatchNorm2d-6           [-1, 64, 32, 32]             128
  BasicBlockKernel-7           [-1, 64, 32, 32]               0
            Conv2d-8           [-1, 64, 32, 32]           4,096
       BatchNorm2d-9           [-1, 64, 32, 32]             128
           Conv2d-10           [-1, 64, 32, 32]           4,096
      BatchNorm2d-11           [-1, 64, 32, 32]             128
 BasicBlockKernel-12           [-1, 64, 32, 32]               0
           Conv2d-13           [-1, 64, 32, 32]           4,096
      BatchNorm2d-14           [-1, 64,

In [237]:
# reducing residual layer, reducing residual block, in planes = 128 and increasing number of chan modification
net6 = ResNet(BasicBlock, [1, 1, 1], [128, 256, 512], [1, 2, 2], 2048, in_planes=128)
net6 = net6.to(device)
print(summary(net6, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           3,456
       BatchNorm2d-2          [-1, 128, 32, 32]             256
            Conv2d-3          [-1, 128, 32, 32]         147,456
       BatchNorm2d-4          [-1, 128, 32, 32]             256
            Conv2d-5          [-1, 128, 32, 32]         147,456
       BatchNorm2d-6          [-1, 128, 32, 32]             256
        BasicBlock-7          [-1, 128, 32, 32]               0
            Conv2d-8          [-1, 256, 16, 16]         294,912
       BatchNorm2d-9          [-1, 256, 16, 16]             512
           Conv2d-10          [-1, 256, 16, 16]         589,824
      BatchNorm2d-11          [-1, 256, 16, 16]             512
           Conv2d-12          [-1, 256, 16, 16]          32,768
      BatchNorm2d-13          [-1, 256, 16, 16]             512
       BasicBlock-14          [-1, 256,

In [238]:
# reducing residual layer, reducing residual block, increasing number of channels, modifying average pool kernel size, in_planes = 128 
net7 = ResNet(BasicBlock, [1, 1, 1], [128, 256, 512], [1, 2, 2], 8192, in_planes=128, avg_pool_size = 2)
net7 = net7.to(device)
print(summary(net7, (3, 32, 32)))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [-1, 128, 32, 32]           3,456
       BatchNorm2d-2          [-1, 128, 32, 32]             256
            Conv2d-3          [-1, 128, 32, 32]         147,456
       BatchNorm2d-4          [-1, 128, 32, 32]             256
            Conv2d-5          [-1, 128, 32, 32]         147,456
       BatchNorm2d-6          [-1, 128, 32, 32]             256
        BasicBlock-7          [-1, 128, 32, 32]               0
            Conv2d-8          [-1, 256, 16, 16]         294,912
       BatchNorm2d-9          [-1, 256, 16, 16]             512
           Conv2d-10          [-1, 256, 16, 16]         589,824
      BatchNorm2d-11          [-1, 256, 16, 16]             512
           Conv2d-12          [-1, 256, 16, 16]          32,768
      BatchNorm2d-13          [-1, 256, 16, 16]             512
       BasicBlock-14          [-1, 256,

### Training and Testing them to compare performances

In [239]:
import os
import time

Change the model below to train each network.

In [240]:
# Specify the model configuration to use
model = net6.to(device)
if device == 'cuda':
    model = torch.nn.DataParallel(model)
    cudnn.benchmark = True

In [241]:
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [242]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=200)

In [243]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


parameters = count_parameters(model)
print(f"{parameters:,}")
assert parameters<5000000

4,911,754


In [244]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


Change the checkpoint name below to save different model checkpoints. We are saving the model state, validation accuracy for each epoch in checkpoints. We are also printing validation loss and accuracy for each epoch.

In [203]:
# Training

def train(trainloader,epoch,net):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()


def test(testloader,epoch,net,save=True):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    if save==True:
        # Save checkpoint.
        acc = 100.*correct/total
        if acc > best_acc:
            print('Saving..')
            state = {
                'model_state': net.state_dict(),
                'acc': acc,
                'epoch': epoch,
                'optimizer_state': optimizer.state_dict()
            }
            if not os.path.isdir('checkpoint'):
                os.mkdir('checkpoint')
            torch.save(state, './checkpoint/ckpt_net6.pth')
            best_acc = acc
    print('Loss: %.3f | Acc: %.3f%% (%d/%d)' % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))

    
full_start_time = time.time()

for epoch in range(start_epoch, start_epoch+30):
    start_time = time.time()
    train(train_loader, epoch, model)
    test(valid_loader, epoch, model) # Validation dataset
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f"Time Elapsed: {epoch_mins}m {epoch_secs}s")
    scheduler.step()

full_end_time = time.time()
full_mins, full_secs = epoch_time(full_start_time, full_end_time)
print(f"Total time elapsed: {full_mins}m {full_secs}s")


Epoch: 0
Saving..
Loss: 1.073 | Acc: 62.460% (3123/5000)
Time Elapsed: 0m 37s

Epoch: 1
Saving..
Loss: 0.836 | Acc: 70.480% (3524/5000)
Time Elapsed: 0m 37s

Epoch: 2
Saving..
Loss: 0.668 | Acc: 77.160% (3858/5000)
Time Elapsed: 0m 38s

Epoch: 3
Loss: 0.800 | Acc: 73.460% (3673/5000)
Time Elapsed: 0m 38s

Epoch: 4
Loss: 0.677 | Acc: 77.120% (3856/5000)
Time Elapsed: 0m 39s

Epoch: 5
Saving..
Loss: 0.546 | Acc: 81.440% (4072/5000)
Time Elapsed: 0m 40s

Epoch: 6
Saving..
Loss: 0.551 | Acc: 81.620% (4081/5000)
Time Elapsed: 0m 40s

Epoch: 7
Saving..
Loss: 0.488 | Acc: 83.600% (4180/5000)
Time Elapsed: 0m 40s

Epoch: 8
Saving..
Loss: 0.440 | Acc: 84.660% (4233/5000)
Time Elapsed: 0m 41s

Epoch: 9
Saving..
Loss: 0.446 | Acc: 85.580% (4279/5000)
Time Elapsed: 0m 40s

Epoch: 10
Saving..
Loss: 0.437 | Acc: 85.700% (4285/5000)
Time Elapsed: 0m 38s

Epoch: 11
Saving..
Loss: 0.396 | Acc: 87.060% (4353/5000)
Time Elapsed: 0m 37s

Epoch: 12
Loss: 0.454 | Acc: 85.760% (4288/5000)
Time Elapsed: 0m 3

## Visualizing Performance of different models

In [249]:
# Loading checkpoints and their models
models = {'ckpt_net6.pth': net6, 'ckpt_net3.pth': net3}
checkpoints = os.listdir('checkpoint/')

for checkpoint in checkpoints:
    if checkpoint[-4:] == '.pth':
        # Loading the model states and optimizer state to predict test accuracy
        check = torch.load('checkpoint/' + checkpoint)
        models[checkpoint].load_state_dict(check['model_state'], strict=False)
        models[checkpoint].eval()
        # optimizer.load_state_dict(check['optimizer_state'], False)
        val_acc = check['acc']
        epoch = check['epoch']

        print("For Checkpoint : ", str(checkpoint))
        print("Best validation accuracy is ",str(val_acc)," at epoch = ",str(epoch))
        print("Test loss and accuracy : ")
        test(testloader, epoch, model, save=False)
        print("\n")

For Checkpoint :  ckpt_net3.pth
Best validation accuracy is  91.16  at epoch =  28
Test loss and accuracy : 
Loss: 2.300 | Acc: 11.920% (1192/10000)


For Checkpoint :  ckpt_net6.pth
Best validation accuracy is  90.64  at epoch =  28
Test loss and accuracy : 
Loss: 2.300 | Acc: 11.920% (1192/10000)




## References

1. https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
2. https://github.com/kuangliu/pytorch-cifar
3. https://gist.github.com/kevinzakka/d33bf8d6c7f06a9d8c76d97a7879f5cb